## Install and Import Dependencies

In [79]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [80]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions"

## MAKE SOME DETECTIONS

In [81]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
         # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [82]:
results.face_landmarks.landmark[0].z


-0.02948199026286602

In [83]:
import csv
import os
import numpy as np

In [84]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [140]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [141]:
landmarks[-5]

'v500'

In [144]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks) #export columns into csv

In [155]:
class_name = "Wakanda Forever"

In [156]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


## 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [157]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [159]:
df = pd.read_csv('coords.csv')

In [160]:
df.head

<bound method NDFrame.head of                class        x1        y1        z1        v1        x2  \
0              Happy  0.611176  0.488615 -1.525906  0.999047  0.646741   
1              Happy  0.604900  0.488313 -1.519031  0.999101  0.641291   
2              Happy  0.600079  0.488378 -1.489210  0.999151  0.637104   
3              Happy  0.598231  0.488584 -1.489739  0.999172  0.635723   
4              Happy  0.597203  0.488569 -1.540208  0.999183  0.634657   
..               ...       ...       ...       ...       ...       ...   
974  Wakanda Forever  0.608730  0.471587 -0.350806  0.999873  0.621622   
975  Wakanda Forever  0.618745  0.476134 -0.469060  0.999718  0.631361   
976  Wakanda Forever  0.626358  0.488343 -0.585194  0.999736  0.642026   
977  Wakanda Forever  0.635485  0.502440 -0.767892  0.999718  0.654304   
978  Wakanda Forever  0.652028  0.528517 -0.758748  0.999403  0.672321   

           y2        z2        v2        x3  ...      z499  v499      x500  \
0  

In [161]:
df.tail

<bound method NDFrame.tail of                class        x1        y1        z1        v1        x2  \
0              Happy  0.611176  0.488615 -1.525906  0.999047  0.646741   
1              Happy  0.604900  0.488313 -1.519031  0.999101  0.641291   
2              Happy  0.600079  0.488378 -1.489210  0.999151  0.637104   
3              Happy  0.598231  0.488584 -1.489739  0.999172  0.635723   
4              Happy  0.597203  0.488569 -1.540208  0.999183  0.634657   
..               ...       ...       ...       ...       ...       ...   
974  Wakanda Forever  0.608730  0.471587 -0.350806  0.999873  0.621622   
975  Wakanda Forever  0.618745  0.476134 -0.469060  0.999718  0.631361   
976  Wakanda Forever  0.626358  0.488343 -0.585194  0.999736  0.642026   
977  Wakanda Forever  0.635485  0.502440 -0.767892  0.999718  0.654304   
978  Wakanda Forever  0.652028  0.528517 -0.758748  0.999403  0.672321   

           y2        z2        v2        x3  ...      z499  v499      x500  \
0  

In [162]:
df[df['class']=='Sad']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
117,Sad,0.599312,0.511001,-1.740777,0.996578,0.640860,0.429814,-1.629541,0.995594,0.664622,...,-0.004788,0.0,0.692289,0.411244,0.018347,0.0,0.701035,0.402576,0.018632,0.0
118,Sad,0.596554,0.512242,-1.733215,0.996628,0.639130,0.429816,-1.624380,0.995583,0.664001,...,-0.005299,0.0,0.690664,0.409858,0.018390,0.0,0.699184,0.401005,0.018837,0.0
119,Sad,0.595668,0.512482,-1.712211,0.996636,0.638693,0.429779,-1.601173,0.995619,0.663821,...,-0.004987,0.0,0.688907,0.410280,0.017299,0.0,0.697364,0.401153,0.017716,0.0
120,Sad,0.593942,0.512464,-1.702640,0.996620,0.637338,0.429617,-1.593746,0.995558,0.662961,...,-0.005084,0.0,0.688348,0.411943,0.016425,0.0,0.696901,0.402476,0.016834,0.0
121,Sad,0.592540,0.512608,-1.676455,0.996568,0.636213,0.429610,-1.571407,0.995409,0.662322,...,-0.005311,0.0,0.688898,0.411914,0.016713,0.0,0.697509,0.402184,0.017176,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,Sad,0.640127,0.515313,-1.216981,0.998960,0.668547,0.445988,-1.121146,0.998864,0.686499,...,-0.001761,0.0,0.697966,0.442722,0.014793,0.0,0.704872,0.433779,0.015372,0.0
297,Sad,0.638695,0.513754,-1.214788,0.999053,0.666832,0.445460,-1.119525,0.998954,0.685278,...,-0.001511,0.0,0.698623,0.443171,0.015236,0.0,0.705537,0.434466,0.015782,0.0
298,Sad,0.638214,0.513341,-1.231593,0.999125,0.666180,0.445404,-1.138495,0.999008,0.684692,...,-0.001727,0.0,0.699074,0.443143,0.015386,0.0,0.706047,0.434376,0.015958,0.0
299,Sad,0.638420,0.512061,-1.227230,0.999193,0.666175,0.444950,-1.134112,0.999063,0.684612,...,-0.001627,0.0,0.698925,0.440489,0.015461,0.0,0.705779,0.431721,0.016005,0.0


In [163]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [165]:
y_test

387         Victorious
653          Thumbs Up
40               Happy
913    Wakanda Forever
766        Thumbs Down
            ...       
218                Sad
581          Thumbs Up
170                Sad
688          Thumbs Up
415         Victorious
Name: class, Length: 294, dtype: object

## 3.2 Train Machine Learning Classification Model

In [166]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [167]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [174]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [169]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [175]:
fit_models['rc'].predict(X_test)


array(['Victorious', 'Thumbs Up', 'Happy', 'Wakanda Forever',
       'Thumbs Down', 'Sad', 'Sad', 'Happy', 'Wakanda Forever', 'Sad',
       'Thumbs Up', 'Wakanda Forever', 'Wakanda Forever', 'Victorious',
       'Wakanda Forever', 'Sad', 'Victorious', 'Thumbs Down', 'Thumbs Up',
       'Thumbs Up', 'Happy', 'Sad', 'Victorious', 'Victorious', 'Sad',
       'Sad', 'Wakanda Forever', 'Thumbs Up', 'Sad', 'Thumbs Up', 'Sad',
       'Victorious', 'Sad', 'Wakanda Forever', 'Sad', 'Sad', 'Sad',
       'Wakanda Forever', 'Happy', 'Thumbs Up', 'Sad', 'Thumbs Up', 'Sad',
       'Thumbs Down', 'Sad', 'Thumbs Up', 'Thumbs Up', 'Wakanda Forever',
       'Thumbs Up', 'Wakanda Forever', 'Victorious', 'Happy',
       'Victorious', 'Happy', 'Sad', 'Thumbs Up', 'Thumbs Up', 'Sad',
       'Wakanda Forever', 'Thumbs Down', 'Thumbs Down', 'Victorious',
       'Thumbs Up', 'Thumbs Up', 'Sad', 'Thumbs Up', 'Wakanda Forever',
       'Sad', 'Thumbs Up', 'Sad', 'Sad', 'Sad', 'Victorious',
       'Thumbs Down', '

## Evaluate and Serialize Model

In [176]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 


In [177]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [178]:
fit_models['rf'].predict(X_test)

array(['Victorious', 'Thumbs Up', 'Happy', 'Wakanda Forever',
       'Thumbs Down', 'Sad', 'Sad', 'Happy', 'Wakanda Forever', 'Sad',
       'Thumbs Up', 'Wakanda Forever', 'Wakanda Forever', 'Victorious',
       'Wakanda Forever', 'Sad', 'Victorious', 'Thumbs Down', 'Thumbs Up',
       'Thumbs Up', 'Happy', 'Sad', 'Victorious', 'Victorious', 'Sad',
       'Sad', 'Wakanda Forever', 'Thumbs Up', 'Sad', 'Thumbs Up', 'Sad',
       'Victorious', 'Sad', 'Wakanda Forever', 'Sad', 'Sad', 'Sad',
       'Wakanda Forever', 'Happy', 'Thumbs Up', 'Sad', 'Thumbs Up', 'Sad',
       'Thumbs Down', 'Sad', 'Thumbs Up', 'Thumbs Up', 'Wakanda Forever',
       'Thumbs Up', 'Wakanda Forever', 'Victorious', 'Happy',
       'Victorious', 'Happy', 'Sad', 'Thumbs Up', 'Thumbs Up', 'Sad',
       'Wakanda Forever', 'Thumbs Down', 'Thumbs Down', 'Victorious',
       'Thumbs Up', 'Thumbs Up', 'Sad', 'Thumbs Up', 'Wakanda Forever',
       'Sad', 'Thumbs Up', 'Sad', 'Sad', 'Sad', 'Victorious',
       'Thumbs Down', '

In [179]:
y_test

387         Victorious
653          Thumbs Up
40               Happy
913    Wakanda Forever
766        Thumbs Down
            ...       
218                Sad
581          Thumbs Up
170                Sad
688          Thumbs Up
415         Victorious
Name: class, Length: 294, dtype: object

In [180]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

## 4. Make Detections with Model

In [181]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [182]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [187]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
          # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
           
         # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Sad [0.06 0.67 0.08 0.14 0.01 0.04]


C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Sad [0.   0.86 0.01 0.11 0.02 0.  ]


C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Victorious [0.03 0.31 0.05 0.03 0.56 0.02]
Victorious [0.02 0.34 0.05 0.02 0.55 0.02]


C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Victorious [0.03 0.28 0.05 0.02 0.6  0.02]
Victorious [0.03 0.31 0.05 0.02 0.57 0.02]


C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Priyanshu Singh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Victorious [0.03 0.29 0.05 0.02 0.58 0.03]
Victorious [0.02 0.31 0.04 0.04 0.57 0.02]


In [186]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))


(517, 211)